# Sheath Expansion Simulations Results - Analysis Round 1
First from a series of notebooks analysing the sheath expansions simulations in a manner similar to Bergmann in his 2002 paper. This one combines IVs together into a single xarray object for easy later manipulation. 

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.xrfuncs as xrf

In [2]:
importlib.reload(fts)

<module 'flopter.core.fitters' from '/home/jleland/coding/projects/flopter/flopter/core/fitters.py'>

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [3]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [4]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [5]:
skippable_scans = {
    'marconi/spice2/sheath_exp/angled',
#     'marconi/spice2/sheath_exp/angled_1',
#     'marconi/spice2/sheath_exp/angled_longtail_10',
#     'marconi/spice2/sheath_exp/angled_recessed',
#     'marconi/spice2/sheath_exp/angled_recessed_1',
#     'marconi/spice2/sheath_exp/angled_semi-recessed',
#     'marconi/spice2/sheath_exp/angled_semi-recessed_1',
#     'marconi/spice2/sheath_exp/flat',
#     'marconi/spice2/sheath_exp/flat_1',
#     'marconi/spice2/sheath_exp/flat_recessed',
#     'marconi/spice2/sheath_exp/flat_recessed_1',
#     'marconi/spice2/sheath_exp/flat_semi-recessed',
#     'marconi/spice2/sheath_exp/flat_semi-recessed_1',
#     'marconi/spice2/sheath_exp/semi-angled',
#     'marconi/spice2/sheath_exp/semi-angled_1',
#     'marconi/spice2/sheath_exp/semi-angled_recessed',
#     'marconi/spice2/sheath_exp/semi-angled_recessed_1',
#     'marconi/spice2/sheath_exp/semi-angled_semi-recessed',
#     'marconi/spice2/sheath_exp/semi-angled_semi-recessed_1',
    'marconi/spice2/sheath_exp_fwp/flat_10',
    'marconi/spice2/sheath_exp_fwp/flat_10_highV',
    'marconi/spice2/sheath_exp_fwp/flat_8-12_highV',

} 
single_sims = set()

In [6]:
non_standard_variables = {'t', 'WallPot', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

In [7]:
all_angles = {'-1.0', '-2.0', '-3.0', '-4.0', '-5.0', '-6.0', '-7.0', '-8.0', 
              '10.0', '12.0', '15.0', '20.0', '30.0'}
# allowed_angles = ['-2.0', '-3.0', '-5.0', '-8.0', '10.0', '30.0']
allowed_angles = all_angles
disallowed_angles = all_angles - allowed_angles

In [23]:
# scans_searchstr = '*/*/new_sheath_exp/*'
# scans_searchstr = '*/*/sheath_exp_fwp/*'

# scans_1_searchstr = '*/*/sheath_exp_fwp/flat_flush_as'
# scans_2_searchstr = '*/*/se_tempscan/*'
scan_searchstrs = [
#     scans_1_searchstr,
#     scans_2_searchstr,
#     '*/*/sheath_exp_hg/flat_flush_*',
    '*/*/sheath_exp_fflwp/*'
#     '*/*/sheath_exp_hg_fflwp/flat_flush*'
#     '*/*/sheath_exp_fwp/flat_flush_wp-2*'
#     '*/*/sheath_exp_fwp/*wp-2*',
#     '*/*/sheath_exp_fwp/flat_recessed_*',
]

# angles_search_str = '/*[!.{yml, inp}]/backup*'
# angles_search_str = '/*[!{.yml, .inp}]'
angles_search_str = '/alpha_yz_*'

all_run_dirs = {}
# scans_1 = glob.glob(scans_1_searchstr)
# scans_2 = glob.glob(scans_2_searchstr)
# scans = set(scans_1) | set(scans_2) - skippable_scans
scans = set().union(*[glob.glob(scan_searchstr) for scan_searchstr in scan_searchstrs]) - skippable_scans
for scan in scans:
    if scan in single_sims:
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = [scan_run for scan_run in glob.glob(scan + angles_search_str) 
                              if scan_run[-4:] not in disallowed_angles]


skippable = {}
scans = list(scans)
scans.sort()


In [24]:
for i, scan in enumerate(scans):
    print(f"[{i}]: {scan}")
    for j, run in enumerate(all_run_dirs[scan]):
        print(f"\t[{i},{j}]: {'/'.join(run.split('/')[-2:])}")

[0]: marconi/spice2/sheath_exp_fflwp/angled_recessed_as
	[0,0]: angled_recessed_as/alpha_yz_-10.0
	[0,1]: angled_recessed_as/alpha_yz_-11.0
	[0,2]: angled_recessed_as/alpha_yz_-12.0
	[0,3]: angled_recessed_as/alpha_yz_-15.0
	[0,4]: angled_recessed_as/alpha_yz_-2.0
	[0,5]: angled_recessed_as/alpha_yz_-20.0
	[0,6]: angled_recessed_as/alpha_yz_-3.0
	[0,7]: angled_recessed_as/alpha_yz_-30.0
	[0,8]: angled_recessed_as/alpha_yz_-4.0
	[0,9]: angled_recessed_as/alpha_yz_-5.0
	[0,10]: angled_recessed_as/alpha_yz_-6.0
	[0,11]: angled_recessed_as/alpha_yz_-7.0
	[0,12]: angled_recessed_as/alpha_yz_-8.0
	[0,13]: angled_recessed_as/alpha_yz_-9.0
[1]: marconi/spice2/sheath_exp_fflwp/flat_flush_as
	[1,0]: flat_flush_as/alpha_yz_-10.0
	[1,1]: flat_flush_as/alpha_yz_-12.0
	[1,2]: flat_flush_as/alpha_yz_-15.0
	[1,3]: flat_flush_as/alpha_yz_-2.0
	[1,4]: flat_flush_as/alpha_yz_-20.0
	[1,5]: flat_flush_as/alpha_yz_-3.0
	[1,6]: flat_flush_as/alpha_yz_-30.0
	[1,7]: flat_flush_as/alpha_yz_-4.0
	[1,8]: flat_flu

---
## Function For Per-run Analysis

In [40]:
spl_path = spice_dir / all_run_dirs[scans[0]][8]
print(spl_path)
spl_backup = list(spl_path.glob('backup*'))[-1]

/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-4.0


In [41]:
splopter =  spl.Splopter(spl_backup, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True)
print(splopter.tdata.dt)

parser = inp.InputParser(input_filename=spl_backup / 'input.inp')
denormaliser = nrm.Denormaliser(dt=splopter.tdata.dt, input_parser=parser) #, dimensions=3)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-4.0/backup_20210214-1611) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


[[0.00033962]]
dx = 1.0, dy = 1.0, dz = 1.0


In [15]:
def homogenise_run(path):
    splopter =  spl.Splopter(path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                             store_dataframe_fl=True)
    splopter.parse_input()
    iv_data, raw_iv_data = splopter.homogenise(reduced_bin_fract=0.1)
    return splopter.iv_df, iv_data

def get_dummy_fit(iv_data, fitter):
    dummy_params = np.nan * np.zeros_like(fitter.default_values)
    dum_fit_data = fd.IVFitData(iv_data['V'], iv_data['I'], np.nan * np.zeros_like(iv_data['I']), 
                                dummy_params, dummy_params, fitter, 
                                sigma=iv_data['sigma'], chi2=np.nan, reduced_chi2=np.nan)
    return dum_fit_data

In [16]:
def straight_iv_fit(iv_data, cutoff=-2):
    fitter = fts.FullIVFitter()
    iv_region = np.where(iv_data['V'] <= -2)
    fit_iv_data = iv.IVData.non_contiguous_trim(iv_data, iv_region)
    try:        
        fit_data = fitter.fit_iv_data(fit_iv_data, sigma=fit_iv_data['sigma'])
    except ValueError:
        print('CAUGHT: straight fit error')
        fit_data = get_dummy_fit(fit_iv_data, fitter)
    
    return fit_data

def experimental_iv_fit(iv_data, cutoff=-2, show_err_fl=True):
    fitter = fts.ExperimentalIVFitter()
    iv_region = np.where(iv_data['V'] <= cutoff)
    fit_iv_data = iv.IVData.non_contiguous_trim(iv_data, iv_region)
    try:        
        fit_data = fitter.fit_iv_data(fit_iv_data, sigma=fit_iv_data['sigma'])
    except ValueError:
        print('CAUGHT: straight fit error')
        if show_err_fl:
            print(e)
        fit_data = get_dummy_fit(fit_iv_data, fitter)
    
    return fit_data
        
def multi_iv_fit(iv_data, sat_region=-6, **kwargs):
    print(iv_data['sigma'])
    try:
        fit_data = iv_data.multi_fit(sat_region=sat_region, **kwargs)
    except ValueError:
        print('CAUGHT: multi fit error')
        if show_err_fl:
            print(e)
        fit_data = get_dummy_fit(iv_data, fts.FullIVFitter())
        
    return fit_data


def norm_ion_fit(iv_data_orig, sigma=None, show_err_fl=False, v_redund=6):
    iv_data = iv_data_orig.copy()
    if sigma is None:
        iv_data['sigma'] = iv_data['sigma'] * 0.5
    else:
        iv_data['sigma'] = sigma
    
    V_f = fts.IVFitter.find_floating_pot(iv_data['V'][:-1], iv_data['I'][:-1])
#     V_f = iv_data.get_vf()
    print(f'v_f = {V_f}')
    
#     sim_ds = sim_ds.sel(voltage=slice(-9.95, V_f))
    iv_region = np.where(iv_data['V'] < (V_f - v_redund))
    fit_iv_data = iv.IVData.non_contiguous_trim(iv_data, iv_region)
    i_fitter = fts.IonCurrentSEFitter()

    try:
        fit_data = i_fitter.fit(np.float_power(np.abs(fit_iv_data['V'] - V_f), .75), 
                                fit_iv_data['I_i'],
                                sigma=fit_iv_data['sigma'])
    except ValueError as e:
        print('CAUGHT: ion fit value error')
        if show_err_fl:
            print(e)
        fit_data = get_dummy_fit(fit_iv_data, i_fitter)
    except RuntimeError as e:
        print('CAUGHT: ion fit runtime error')
        if show_err_fl:
            print(e)
        fit_data = get_dummy_fit(fit_iv_data, i_fitter)
        
    return fit_data

def norm_full_fit(iv_data, show_err_fl=False):
#     V_f = iv_data.get_vf()
    V_f = fts.IVFitter.find_floating_pot(iv_data['V'][:-1], iv_data['I'][:-1])
    
    iv_region = np.where(iv_data['V'] < V_f)
    fit_iv_data = iv.IVData.non_contiguous_trim(iv_data, iv_region)
    
    voltage = fit_iv_data['V'] - V_f
    current = fit_iv_data['I']
    d_current = fit_iv_data['sigma']
    
    iv_fitter = fts.NormalisedIVFitter()
    try:
        fit_data = iv_fitter.fit(voltage, current, sigma=d_current)
    except ValueError as e:
        print('CAUGHT: norm full fit error')
        if show_err_fl:
            print(e)
        fit_data = get_dummy_fit(fit_iv_data, iv_fitter)
        
    return fit_data
    
def norm_electron_fit(iv_data, sigma=None, show_err_fl=False):
    V_f = fts.IVFitter.find_floating_pot(iv_data['V'][:-1], iv_data['I'][:-1])
    
    if sigma is None:
        iv_data['sigma'] = iv_data['sigma'] * 0.5
    else:
        iv_data['sigma'] = sigma
    
    iv_region = np.where(iv_data['V'] < V_f)
    fit_iv_data = iv.IVData.non_contiguous_trim(iv_data, iv_region)
    
    voltage = fit_iv_data['V']
    current = fit_iv_data['I_e']
    d_current = fit_iv_data['sigma']
    
    elec_fitter = fts.ElectronCurrentFitter()
    try:
        fit_data = elec_fitter.fit(voltage, current, sigma=d_current)
    except ValueError as e:
        print('CAUGHT: electron fit error')
        if show_err_fl:
            print(e)
        fit_data = get_dummy_fit(fit_iv_data, elec_fitter)
        
    return fit_data

In [42]:
# Test
iv_df, iv_data = homogenise_run(spl_backup)
# iv_df = iv_df.drop(iv_df.index[-1])
iv_df

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-4.0/backup_20210214-1611) doesn't seem to be valid.
Continuing anyway.


,voltage,voltage_wall,current,current_e,current_i,d_current,d_current_e,d_current_i,time
799,-10.000,-4.149440,-21.378942,-0.021058,21.400000,0.488658,0.014811,0.487331,25.778496
800,-9.975,-4.207024,-21.747340,-0.105292,21.852632,0.560949,0.035023,0.558547,25.810759
801,-9.950,-4.166443,-21.410512,-0.052646,21.463158,0.495104,0.023038,0.492908,25.843023
802,-9.925,-4.211004,-20.894714,-0.084233,20.978947,0.455027,0.028651,0.452684,25.875286
803,-9.900,-4.236249,-21.021033,-0.073704,21.094737,0.490442,0.026954,0.490919,25.907550
...,...,...,...,...,...,...,...,...,...
1193,-0.150,-4.556808,177.966111,-185.050322,7.084211,1.361038,1.366987,0.290060,38.490295
1194,-0.125,-4.577686,179.408849,-187.377270,7.968421,1.365747,1.382994,0.276133,38.522558
1195,-0.100,-4.573857,182.062108,-189.683161,7.621053,1.359358,1.298992,0.304935,38.554822
1196,-0.075,-4.579229,182.978055,-190.262265,7.284211,1.282859,1.289716,0.263350,38.587085


In [43]:
iv_df.loc[iv_df['voltage'] > -10]


,voltage,voltage_wall,current,current_e,current_i,d_current,d_current_e,d_current_i,time
799,-10.000,-4.149440,-21.378942,-0.021058,21.400000,0.488658,0.014811,0.487331,25.778496
800,-9.975,-4.207024,-21.747340,-0.105292,21.852632,0.560949,0.035023,0.558547,25.810759
801,-9.950,-4.166443,-21.410512,-0.052646,21.463158,0.495104,0.023038,0.492908,25.843023
802,-9.925,-4.211004,-20.894714,-0.084233,20.978947,0.455027,0.028651,0.452684,25.875286
803,-9.900,-4.236249,-21.021033,-0.073704,21.094737,0.490442,0.026954,0.490919,25.907550
...,...,...,...,...,...,...,...,...,...
1193,-0.150,-4.556808,177.966111,-185.050322,7.084211,1.361038,1.366987,0.290060,38.490295
1194,-0.125,-4.577686,179.408849,-187.377270,7.968421,1.365747,1.382994,0.276133,38.522558
1195,-0.100,-4.573857,182.062108,-189.683161,7.621053,1.359358,1.298992,0.304935,38.554822
1196,-0.075,-4.579229,182.978055,-190.262265,7.284211,1.282859,1.289716,0.263350,38.587085


In [44]:
fig, ax = plt.subplots(2)

iv_data.plot(ax=ax[0])

# norm_ion_fit(iv_data, sigma=iv_df['d_current_i'].values, show_err_fl=True, v_redund=0).plot(ax=ax[1])
norm_ion_fit(iv_data, sigma=iv_df['d_current_i'].values, show_err_fl=True, v_redund=4).plot(ax=ax[1])
xrf.norm_ion_fit(iv_data, sigma=iv_df['d_current_i'].values, show_err_fl=True, v_redund=6, voltage_cap=-15.0).plot(ax=ax[1])

v_f = -4.253915436559333
CAUGHT: ion fit value error
`ydata` must not be empty!


In [45]:
fig, ax = plt.subplots()

a_vals = []
d_a_vals = []
v_caps = np.linspace(-30.0, -4.0, 101)

for v_cap in v_caps:
    f_data = xrf.norm_ion_fit(iv_data, sigma=iv_df['d_current_i'].values, show_err_fl=True, v_redund=1.5, voltage_cap=v_cap)
    a_vals.append(f_data.get_sheath_exp())
    d_a_vals.append(f_data.get_sheath_exp_err())
ax.errorbar(v_caps, a_vals, yerr=d_a_vals)
ax.set_ylim(0.0, 0.2)

CAUGHT: ion fit runtime error
Optimal parameters not found: The maximum number of function evaluations is exceeded.
CAUGHT: ion fit value error
`ydata` must not be empty!
CAUGHT: ion fit value error
`ydata` must not be empty!
CAUGHT: ion fit value error
`ydata` must not be empty!
CAUGHT: ion fit value error
`ydata` must not be empty!
CAUGHT: ion fit value error
`ydata` must not be empty!
CAUGHT: ion fit value error
`ydata` must not be empty!
CAUGHT: ion fit value error
`ydata` must not be empty!


(0.0, 0.2)

In [46]:
fig, ax = plt.subplots()

xrf.norm_ion_fit(iv_data, sigma=iv_df['d_current_i'].values, show_err_fl=True, v_redund=0).plot(ax=ax)
xrf.norm_ion_fit(iv_data, sigma=iv_df['d_current_i'].values, show_err_fl=True, v_redund=1.5, 
                 voltage_cap=-10.0).plot(ax=ax)
# xrf.norm_ion_fit(iv_data, sigma=iv_df['d_current_i'].values, show_err_fl=True, v_redund=6, 
#                  voltage_cap=-15.0).plot(ax=ax)

In [32]:
# iv.IVData.fractional_trim(iv_data, trim_beg=0.5).plot(ax=ax) #.multi_fit().plot(ax=ax)

fig, ax = plt.subplots(2)
# print(straight_iv_fit(iv_data).to_dict())
# multi_iv_fit(iv_data, plot_fl=True, mode=5, trimming_vals=(0.2, 0.0, 0.001), minimise_temp_fl=False).plot(ax=ax)
fit_data = norm_full_fit(iv_data, show_err_fl=True)
fit_data.plot(ax=ax[0])
norm_ion_fit(iv_data, sigma=iv_df['d_current_i'], show_err_fl=True).plot(ax=ax[1])


v_f = -3.046896633701932


In [38]:
importlib.reload(xrf)

<module 'flopter.spice.xrfuncs' from '/home/jleland/coding/projects/flopter/flopter/spice/xrfuncs.py'>

In [39]:
iparams = xrf.get_initial_params(20.0)
print(iparams)
xrf.straight_iv_fit(iv_data, all_initial_values=iparams).plot()

{'temperature': 1.0, 'floating_potential': -2.5, 'isat': array([4.5507673]), 'sheath_exp_param': 0.01}


In [33]:
fig, ax = plt.subplots(2)

# iv_df['current_e'].plot.line(x='voltage', ax=ax[0])
ax[0].plot(iv_data['V'], iv_data['I_e'])
e_fitter = fts.ElectronCurrentFitter()
e_fitter.fit(iv_data['V'], iv_data['I_e']).plot(ax=ax[0])

v_dum = np.linspace(-30,0)
e_current = -21.818 * np.exp(v_dum+2.4398)
ax[0].plot(v_dum, e_current)

ese_fitter = fts.ElectronCurrentSEFitter()
# ese_fitter.fit(iv_data['V'], iv_data['I_e'], initial_vals=[1.0,-2.5,-21.818,0.006]).plot(ax=ax[1])
norm_electron_fit(iv_data).plot(ax=ax[1])



# log-plotting of IV
fig, ax = plt.subplots()
# ax.plot(v_dum, ese_fitter.fit_function(v_dum, *[1.0,-2.5,-21.818,0.006]))
# log_slice = slice(800,1000)
log_slice = np.where(np.isfinite(np.log( -iv_data['I_e'])))

ax.plot(iv_data['V'],np.log( -iv_data['I_e']))
ax.plot(iv_data['V'][log_slice],np.log( -iv_data['I_e'][log_slice]))

sl_fitter = fts.StraightLineFitter()
sl_fit_data = sl_fitter.fit(iv_data['V'][log_slice],np.log(-iv_data['I_e'][log_slice]))
sl_fit_data.plot(ax=ax)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log


In [34]:
fig, ax = plt.subplots(3)
xrf.multi_iv_fit(iv_data).plot(ax=ax[0])
xrf.experimental_iv_fit(iv_data).plot(ax=ax[1])
xrf.multi_iv_fit(iv_data, iv_fitter=fts.ExperimentalIVFitter()).plot(ax=ax[2])
# print(iv_df['d_current_e'])

In [17]:
fit_data = iv_data.multi_fit(sat_region=-6, trimming_vals=[0.1,0.1,0.02], plot_fl=True)

/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:749: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.
  func(*args)
/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:1705: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.
  return self.func(*args)


In [28]:
importlib.reload(iv)
importlib.reload(xrf)

<module 'flopter.spice.xrfuncs' from '/home/jleland/coding/projects/flopter/flopter/spice/xrfuncs.py'>

In [48]:
fig, ax = plt.subplots(2)

norm_iv_data = iv_data.copy()
norm_iv_data['I'] = denormaliser(norm_iv_data['I'], 'current') 
norm_iv_data['I_i'] = denormaliser(norm_iv_data['I_i'], 'current') 
norm_iv_data['I_e'] = denormaliser(norm_iv_data['I_e'], 'current') 

norm_ion_fit(iv_data, sigma=iv_df['d_current_i'], show_err_fl=True, v_redund=4).plot(ax=ax[0])
norm_ion_fit(norm_iv_data, sigma=denormaliser(iv_df['d_current_i'].values, 'current') , show_err_fl=True).plot(ax=ax[1])

v_f = -13.575000000000001
v_f = -13.575000000000001


In [18]:
splopter.parse_input()
splopter.plot_2d_variable()

AttributeError: 'Spice2TData' object has no attribute 'by'

## Run analysis on subset of runs
To see how to combine them into a dataset

In [23]:
# import importlib
# importlib.reload(iv)
# importlib.reload(spl)

In [15]:
for scan in scans:
    for run_dir in all_run_dirs[scan]:
        run_path = spice_dir / run_dir
        bups = list(run_path.glob('backup*'))
        bups.sort()
        final_state = bups[-1].name
        
        print(run_dir)

marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-10.0
marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-12.0
marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-2.0
marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-20.0
marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-3.0
marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-30.0
marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-4.0
marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-5.0
marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-6.0
marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-8.0
marconi/spice2/sheath_exp_hg/flat_flush_gapfill/alpha_yz_-11.0
marconi/spice2/sheath_exp_hg/flat_flush_gapfill/alpha_yz_-15.0
marconi/spice2/sheath_exp_hg/flat_flush_gapfill/alpha_yz_-6.0
marconi/spice2/sheath_exp_hg/flat_flush_gapfill/alpha_yz_-7.0
marconi/spice2/sheath_exp_hg/flat_flush_gapfill/alpha_yz_-9.0


In [16]:
iv_dfs = {}
fit_datas = {}
iv_datas = {}
failed_runs = []

In [17]:
scans_oi = scans[:2]

for scan in scans:
    for run_dir in all_run_dirs[scan]:
        run_path = spice_dir / run_dir
        
        bups = list(run_path.glob('backup*'))
        bups.sort()
        final_state_path = bups[-1]
        
        print(f'\n\n --- {run_dir} --- \n')
        if run_dir in iv_dfs:
            print("skipping...")
            continue
        
#         if '-2.0' in run_dir: # or '-3.0' in run_dir:
#             print(f'!! Skipping {run_dir}...')
#             continue
        
        try:
#         iv_df, fit_data = analyse_run(run_path, mode=5, trimming_vals=(0.2, 0.0, 0.001))
            iv_df, iv_data = homogenise_run(final_state_path) #, minimise_temp_fl=False)

            iv_dfs[run_dir] = iv_df
            iv_datas[run_dir] = iv_data
        except Exception as e:
            failed_runs.append(run_dir)
            print(f'Failed on {run_dir}, {e}')



 --- marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-10.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-10.0/backup_20201222-0127) doesn't seem to be valid.
Continuing anyway.


 --- marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-12.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-12.0/backup_20201221-0640) doesn't seem to be valid.
Continuing anyway.


 --- marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-2.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-2.0/backup_20210116-1340) doesn't seem to be valid.
Continuing anyway.


 --- marconi/spice2/sheath_exp_hg/flat_flush_as/a

In [18]:
failed_runs

[]

In [23]:
# testing an individual problem path

# p = pth.Path("/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/flat_recessed/alpha_yz_-1.0/backup_20200810-0213")
# p = pth.Path("/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/flat_recessed/alpha_yz_-1.0/backup_20200809-1816")
# splopter =  spl.Splopter(p, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
#                          store_dataframe_fl=True)
# splopter.parse_input()
# iv_data, raw_iv_data = splopter.homogenise()
# splopter.iv_df

In [24]:
# fig, ax = plt.subplots()

# for i in range(len(splopter.tdata.objectscurrente)):
#     ax.plot(splopter.tdata.objectscurrente[i][::100], label=i)
#     ax.legend()

In [19]:
def fd_to_ds(fit_data, fit_prefix='fit', extras=None):
    fit_ds = xr.Dataset(fit_data.to_dict()).drop(['raw_y', 'sigma', 'fit_y'])
    if extras is not None:
        for label, value in extras.items():
            fit_ds[label] = value
    fit_ds = fit_ds.assign(voltage_min=fit_ds.raw_x.min(), voltage_max=fit_ds.raw_x.max()).drop('raw_x').expand_dims(dim=['theta'])
    fit_ds = fit_ds.rename_vars({data_var: f'{fit_prefix}_{data_var}' for data_var in fit_ds.data_vars})
    
    return fit_ds

In [22]:
datasets = {}
probes = []
thetas = []


for scan in scans:
    for run_dir in all_run_dirs[scan]:
        if run_dir in failed_runs or run_dir not in iv_datas:
            print(f'Skipping {run_dir} as it failed...')
            continue
        fit_dss = []
        angle = float(run_dir.split('/')[-1].split('-')[-1])
#         probe = run_dir.split('/')[-2].split('_1')[0]
        probe = '_'.join(run_dir.split('/')[-2].split('_')[0:2])
        print(probe, angle)
        
#         if angle == 2.0:
#             continue
        
        if angle not in thetas:
            thetas.append(angle)
        if probe not in probes:
            probes.append(probe)
        if probe not in datasets:
            datasets[probe] = []

        iv_data = iv_datas[run_dir]
        iv_df = iv_dfs[run_dir]
        
        iv_ds = iv_df.to_xarray().swap_dims({'index':'voltage'}).drop('index').expand_dims(dim=['theta'])
        
        
        str_iv_fit_ds = fd_to_ds(straight_iv_fit(iv_data), fit_prefix='str_iv', extras={'run_dir': run_dir})
#         multi_fit_ds = fd_to_ds(multi_iv_fit(iv_data), fit_prefix='multi')
        norm_iv_fit_ds = fd_to_ds(norm_full_fit(iv_data), fit_prefix='norm_iv')
        ion_fit_ds = fd_to_ds(norm_ion_fit(iv_data, sigma=iv_df['d_current_i']), fit_prefix='ion')
#         fit_ds = fd_to_ds(fit_datas[run_dir], fit_prefix='fit', )
        
        ds = xr.merge([str_iv_fit_ds, norm_iv_fit_ds, ion_fit_ds, iv_ds]).assign_coords(
            theta=xr.DataArray([angle], dims='theta')
        )
        datasets[probe].append(ds)        


flat_flush 10.0
v_f = 0.025000000000001663
flat_flush 12.0
v_f = -3.541231005117194
flat_flush 15.0
v_f = -3.7351471614157923
flat_flush 2.0
v_f = -1.0
flat_flush 20.0
v_f = -3.19025226093094
flat_flush 3.0
v_f = -4.533295594305094
flat_flush 30.0
v_f = -2.8910009478651117
flat_flush 4.0
v_f = -4.507693311562478
flat_flush 5.0
v_f = -4.223083125972569
flat_flush 6.0
v_f = -4.228695108836053
flat_flush 8.0
v_f = -3.887075223639321


flat_recessed 1.0
- CAUGHT: straight fit error
- CAUGHT: norm full fit error
- CAUGHT: ion fit value error

flat_recessed 2.0
- CAUGHT: norm full fit error
- CAUGHT: ion fit value error

flat_recessed 3.0
- CAUGHT: ion fit runtime error

flat_semi-recessed 1.0
- CAUGHT: norm full fit error
- CAUGHT: ion fit value error

In [61]:
# iv_ds = iv_df.to_xarray().swap_dims({'index':'voltage'}).drop('index').expand_dims(dim=['angle', 'probe'])

# fit_ds = xr.Dataset(fit_data.to_dict()).drop(['raw_y', 'sigma', 'fit_y'])
# fit_ds = fit_ds.assign(voltage_min=fit_ds.raw_x.min(), voltage_max=fit_ds.raw_x.max()).drop('raw_x').expand_dims(dim=['angle', 'probe'])
# fit_ds = fit_ds.rename_vars({data_var: f'fit_{data_var}' for data_var in fit_ds.data_vars})

# ds = xr.merge([fit_ds, iv_ds]).assign_coords(
#     angle=xr.DataArray([angle], dims='angle'), 
#     probe=xr.DataArray([probe], dims='probe')
# )
# ds

In [27]:
probes

['flat_flush']

In [28]:
# probes_list = list(probes)
for i, probe in enumerate(probes):
    print(i, probe)

print()

for i, theta in enumerate(thetas):
    print(i, theta)

    

0 flat_flush

0 10.0
1 12.0
2 15.0
3 2.0
4 20.0
5 3.0
6 30.0
7 4.0
8 5.0
9 6.0
10 8.0


In [40]:
for probe, probe_list in datasets.items():
    print(probe)

flat_flush


In [41]:
for ds in datasets['angled_flush_as']:
    print(ds.theta)

KeyError: 'angled_flush_as'

### Different DataArrays to concatenate all of the datasets in different ways

for 3x3 and 2x2 scans

In [31]:
# 3x3 sims

# d_perps = xr.DataArray(
#     [0.0, 5.0e-4, 1.0e-3], 
#     dims=['recession'],
# ).assign_coords(recession_desc=('recession', ['flush', 'semi-recessed', 'recessed']))

# theta_p = xr.DataArray(
#     [0.0, 5, 10.0],
#     dims='theta_p'
# ).assign_coords(theta_p_desc=('theta_p', ['flat', 'semi-angled', 'angled']))


In [49]:
# 2x2 sims

# d_perps = xr.DataArray(
#     [0.0, 1.0e-3], 
#     dims='recession',
#     coords={'recession':[0.0, 1.0e-3]}
# ).assign_coords(recession_desc=('recession', ['flush', 'recessed']))

# theta_p = xr.DataArray(
#     [0.0, 10.0],
#     dims='theta_p',
#     coords={'theta_p':[0.0, 10.0]}
# ).assign_coords(theta_p_desc=('theta_p', ['flat', 'angled']))


In [51]:
# for 2x2 and 3x3

theta_datasets = []
for probe, probe_list in datasets.items():
#     theta_da = xr.DataArray(thetas, dims=['theta'])
    theta_combined_ds = xr.concat(probe_list, dim='theta').sortby('theta')
#     .assign_coords(
#         probe=xr.DataArray([d_perp], dims='recession')
#     )
    theta_datasets.append(theta_combined_ds)
    
# theta_p_datasets = [
#     xr.concat([theta_datasets[3], theta_datasets[6], theta_datasets[0]], dim=theta_p),
#     xr.concat([theta_datasets[5], theta_datasets[8], theta_datasets[2]], dim=theta_p),
#     xr.concat([theta_datasets[4], theta_datasets[7], theta_datasets[1]], dim=theta_p),
# ]
theta_p_datasets = [
    xr.concat([theta_datasets[2], theta_datasets[0]], dim=theta_p),
    xr.concat([theta_datasets[4], theta_datasets[1]], dim=theta_p),
]

    
# probe_da = xr.DataArray(list(probes), dims=['probe'])
# combined_ds = xr.concat(theta_datasets, dim='probe')
# combined_ds = xr.combine_nested(datasets_nl, concat_dim=['probe', 'theta'])
combined_ds = xr.concat(theta_p_datasets, dim=d_perps)
combined_ds

<xarray.Dataset>
Dimensions:               (recession: 2, theta: 9, theta_p: 2, voltage: 1204)
Coordinates:
  * recession             (recession) float64 0.0 0.001
    theta_p_desc          (theta_p) <U6 'flat' 'angled'
  * theta_p               (theta_p) float64 0.0 10.0
  * voltage               (voltage) float64 -29.97 -29.95 -29.93 ... 0.075 0.1
  * theta                 (theta) float64 3.0 4.0 5.0 6.0 ... 12.0 20.0 30.0
    recession_desc        (recession) <U8 'flush' 'recessed'
Data variables:
    str_iv_chi2           (recession, theta_p, theta) float64 6.647e+03 ... 1.265e+03
    str_iv_reduced_chi2   (recession, theta_p, theta) float64 5.962 ... 1.135
    str_iv_I_sat          (recession, theta_p, theta) float64 -10.62 ... -83.3
    str_iv_d_I_sat        (recession, theta_p, theta) float64 0.07518 ... 0.2509
    str_iv_a              (recession, theta_p, theta) float64 0.004903 ... 0.00885
    str_iv_d_a            (recession, theta_p, theta) float64 0.001219 ... 0.0003828
    str_iv_T_e            (recession, theta_p, theta) float64 1.429 ... 1.03
    str_iv_d_T_e          (recession, theta_p, theta) float64 0.01188 ... 0.0101
    str_iv_V_f            (recession, theta_p, theta) float64 -4.493 ... -2.742
    str_iv_d_V_f          (recession, theta_p, theta) float64 0.0113 ... 0.005157
    str_iv_run_dir        (recession, theta_p, theta) <U63 'marconi/spice2/sheath_exp_fwp/flat_flush_as/alpha_yz_-3.0' ... 'marconi/spice2/sheath_exp_fwp/angled_recessed_as/alpha_yz_-30.0'
    str_iv_voltage_min    (recession, theta_p, theta) float64 -29.97 ... -29.97
    str_iv_voltage_max    (recession, theta_p, theta) float64 -2.025 ... -2.025
    norm_iv_chi2          (recession, theta_p, theta) float64 780.1 ... 1.245e+03
    norm_iv_reduced_chi2  (recession, theta_p, theta) float64 0.7641 ... 1.145
    norm_iv_I_sat         (recession, theta_p, theta) float64 -9.839 ... -83.01
    norm_iv_d_I_sat       (recession, theta_p, theta) float64 0.0216 ... 0.2106
    norm_iv_a             (recession, theta_p, theta) float64 0.01325 ... 0.00909
    norm_iv_d_a           (recession, theta_p, theta) float64 0.0003258 ... 0.0003525
    norm_iv_voltage_min   (recession, theta_p, theta) float64 -25.57 ... -27.23
    norm_iv_voltage_max   (recession, theta_p, theta) float64 -0.02284 ... -0.00072
    ion_chi2              (recession, theta_p, theta) float64 556.5 ... 1.052e+03
    ion_reduced_chi2      (recession, theta_p, theta) float64 0.6463 ... 1.135
    ion_I_sat             (recession, theta_p, theta) float64 9.791 ... 83.27
    ion_d_I_sat           (recession, theta_p, theta) float64 0.02519 ... 0.2566
    ion_a                 (recession, theta_p, theta) float64 0.01389 ... 0.008704
    ion_d_a               (recession, theta_p, theta) float64 0.0003671 ... 0.0004109
    ion_voltage_min       (recession, theta_p, theta) float64 2.841 ... 2.829
    ion_voltage_max       (recession, theta_p, theta) float64 11.37 ... 11.92
    current               (recession, theta_p, theta, voltage) float64 -11.6 ... 921.0
    current_e             (recession, theta_p, theta, voltage) float64 0.0 ... -998.8
    current_i             (recession, theta_p, theta, voltage) float64 11.6 ... 77.77
    d_current             (recession, theta_p, theta, voltage) float64 0.315 ... 14.49
    d_current_e           (recession, theta_p, theta, voltage) float64 0.0 ... 14.37
    d_current_i           (recession, theta_p, theta, voltage) float64 0.315 ... 2.228
    time                  (recession, theta_p, theta, voltage) float64 0.0 ... 5.307
    None                  (recession) float64 0.0 0.001

For non-square scans, just concat along a 'probe' dimension which holds the necessary information

In [65]:
probes = ['flat_flush_gapless', 'flat_flush', 'angled_flush', 'flat_recessed', 'angled_recessed']

recession_vals = [0.0, 0.0, 0.0, 1.0e-3, 1.0e-3]
recession_descs = ['flush', 'flush', 'flush', 'recessed', 'recessed']
recession_flags = [False, False, False, True, True]

theta_p_vals = [0.0, 0.0, 10.0, 0.0, 10.0]
theta_p_descs = ['flat', 'flat', 'angled', 'flat', 'angled']
theta_p_flags = [False, False, True, False, True]

gap_vals = [0.0, 1.0e-3, 1.0e-3, 1.0e-3, 1.0e-3]
gap_descs = ['gapless', '', '', '', '']

probe_da = xr.DataArray(
    probes,
    dims='probe',
    coords={'probe': probes}
).assign_coords(
    recession=('probe', recession_vals), 
    recession_desc=('probe', recession_descs), 
    recessed_fl=('probe', recession_flags),
    theta_p=('probe', theta_p_vals),
    theta_p_desc=('probe', theta_p_descs),
    angled_fl=('probe', theta_p_flags),
    gap=('probe', gap_vals),
    gap_desc=('probe', gap_descs),
)

probe_da.sel(probe='flat_flush_gapless')

<xarray.DataArray ()>
array('flat_flush_gapless', dtype='<U18')
Coordinates:
    probe           <U18 'flat_flush_gapless'
    recession       float64 0.0
    recession_desc  <U8 'flush'
    recessed_fl     bool False
    theta_p         float64 0.0
    theta_p_desc    <U6 'flat'
    angled_fl       bool False
    gap             float64 0.0
    gap_desc        <U7 'gapless'

In [66]:

thetascan_datasets = {}
for probe, probe_list in datasets.items():
    theta_combined_ds = xr.concat(probe_list, dim='theta').sortby('theta')

    thetascan_datasets[probe] = theta_combined_ds
    
thetascan_dss_toconcat = [
    thetascan_datasets['flat_flush_gapless_as'], 
    thetascan_datasets['flat_flush_as'],
    thetascan_datasets['angled_flush_as'],
    thetascan_datasets['flat_recessed_as'],
    thetascan_datasets['angled_recessed_as'],
]

combined_ds = xr.concat(thetascan_dss_toconcat, dim=probe_da)
combined_ds

KeyError: 'flat_flush_gapless_as'

## Alternative probe-wise concatenation for tempscan

In [34]:
probes = ['flat_flush_as', 'flat_flush_T10', 'flat_flush_T20', 'flat_flush_T50']

recession_vals = [0.0, 0.0, 0.0, 0.0]
recession_descs = ['flush', 'flush', 'flush', 'flush']
recession_flags = [False, False, False, False]

theta_p_vals = [0.0, 0.0, 0.0, 0.0]
theta_p_descs = ['flat', 'flat', 'flat', 'flat']
theta_p_flags = [False, False, False, False]

temp_vals = [5.0, 10.0, 20.0, 50.0]

temp_da = xr.DataArray(
    temp_vals,
    dims='temperature',
    coords={'temperature': temp_vals}
).assign_coords(
    probe=('temperature', probes),
    recession=('temperature', recession_vals), 
    recession_desc=('temperature', recession_descs), 
    recessed_fl=('temperature', recession_flags),
    theta_p=('temperature', theta_p_vals),
    theta_p_desc=('temperature', theta_p_descs),
    angled_fl=('temperature', theta_p_flags),
)

temp_da.sel(temperature=slice(5,10))

<xarray.DataArray (temperature: 2)>
array([ 5., 10.])
Coordinates:
  * temperature     (temperature) float64 5.0 10.0
    probe           (temperature) <U14 'flat_flush_as' 'flat_flush_T10'
    recession       (temperature) float64 0.0 0.0
    recession_desc  (temperature) <U5 'flush' 'flush'
    recessed_fl     (temperature) bool False False
    theta_p         (temperature) float64 0.0 0.0
    theta_p_desc    (temperature) <U4 'flat' 'flat'
    angled_fl       (temperature) bool False False

In [35]:

thetascan_datasets = {}
for probe, probe_list in datasets.items():
    theta_combined_ds = xr.concat(probe_list, dim='theta').sortby('theta')

    thetascan_datasets[probe] = theta_combined_ds
    
thetascan_dss_toconcat = [
    thetascan_datasets['flat_flush_as'],
    thetascan_datasets['flat_flush_T10'],
]
thetascan_dss_toconcat_lowV = [
    thetascan_datasets['flat_flush_T20'],
    thetascan_datasets['flat_flush_T50'],
]

highV_ds = xr.concat(thetascan_dss_toconcat, dim=temp_da.sel(temperature=slice(5,10)))
lowV_ds = xr.concat(thetascan_dss_toconcat_lowV, dim=temp_da.sel(temperature=slice(20,50)))

In [23]:
combined_ds = xr.concat(datasets['flat_flush'], dim='theta').sortby('theta')
combined_ds

<xarray.Dataset>
Dimensions:               (theta: 11, voltage: 1204)
Coordinates:
  * voltage               (voltage) float64 -29.97 -29.95 -29.93 ... 0.075 0.1
  * theta                 (theta) float64 2.0 3.0 4.0 5.0 ... 15.0 20.0 30.0
Data variables:
    str_iv_chi2           (theta) float64 488.0 4.618e+03 ... 1.55e+03 1.197e+03
    str_iv_reduced_chi2   (theta) float64 0.5824 4.142 5.07 ... 1.392 1.074
    str_iv_I_sat          (theta) float64 -6.478 -10.55 -13.83 ... -44.14 -70.85
    str_iv_d_I_sat        (theta) float64 0.1061 0.06414 ... 0.1698 0.2297
    str_iv_a              (theta) float64 0.0297 0.006142 ... 0.0176 0.01018
    str_iv_d_a            (theta) float64 0.01001 0.001074 ... 0.0004292
    str_iv_T_e            (theta) float64 2.725 1.491 1.448 ... 0.9667 1.098
    str_iv_d_T_e          (theta) float64 1.246 0.01105 ... 0.00767 0.01038
    str_iv_V_f            (theta) float64 0.3754 -4.454 -4.473 ... -3.33 -2.854
    str_iv_d_V_f          (theta) float64 3.899 0.009809 ... 0.006023 0.0052
    str_iv_run_dir        (theta) <U66 'marconi/spice2/sheath_exp_fwp/flat_flush_wp-2_as_1_/alpha_yz_-2.0' ... 'marconi/spice2/sheath_exp_fwp/flat_flush_wp-2_as_1_/alpha_yz_-30.0'
    str_iv_voltage_min    (theta) float64 -29.97 -29.97 -29.97 ... -29.97 -29.97
    str_iv_voltage_max    (theta) float64 -8.95 -2.025 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (theta) float64 506.1 919.6 ... 1.562e+03 1.175e+03
    norm_iv_reduced_chi2  (theta) float64 0.6025 0.9052 1.042 ... 1.461 1.087
    norm_iv_I_sat         (theta) float64 -6.366 -9.909 -12.99 ... -43.61 -70.15
    norm_iv_d_I_sat       (theta) float64 0.02114 0.02343 ... 0.1544 0.1886
    norm_iv_a             (theta) float64 0.01657 0.01229 ... 0.01958 0.01072
    norm_iv_d_a           (theta) float64 0.0004216 0.0003495 ... 0.0003763
    norm_iv_voltage_min   (theta) float64 -28.97 -25.44 -25.47 ... -26.78 -27.08
    norm_iv_voltage_max   (theta) float64 -7.95 -0.0167 ... -0.009748 -0.008999
    ion_chi2              (theta) float64 500.7 483.7 593.5 ... 879.8 854.9
    ion_reduced_chi2      (theta) float64 0.5961 0.6234 0.7648 ... 1.061 1.016
    ion_I_sat             (theta) float64 6.384 9.849 12.84 ... 44.48 70.74
    ion_d_I_sat           (theta) float64 0.02101 0.03081 0.045 ... 0.2001 0.278
    ion_a                 (theta) float64 0.01625 0.01313 ... 0.01693 0.009697
    ion_d_a               (theta) float64 0.0004169 0.0004285 ... 0.0005056
    ion_voltage_min       (theta) float64 4.735 3.842 3.842 ... 3.838 3.838
    ion_voltage_max       (theta) float64 12.49 11.33 11.34 ... 11.77 11.87
    voltage_wall          (theta, voltage) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    current               (theta, voltage) float64 -7.857 -7.984 ... 740.5 749.4
    current_e             (theta, voltage) float64 0.0 0.0 0.0 ... -806.0 -812.2
    current_i             (theta, voltage) float64 7.857 7.984 ... 65.46 62.85
    d_current             (theta, voltage) float64 0.2108 0.1932 ... 7.615 9.685
    d_current_e           (theta, voltage) float64 0.0 0.0 0.0 ... 7.659 9.125
    d_current_i           (theta, voltage) float64 0.2108 0.1932 ... 2.043 2.258
    time                  (theta, voltage) float64 0.0 0.06419 ... 5.302 5.307

## Add Probe Data

In [24]:
# combined_ds['theta_p'] = theta_p
combined_ds['theta_p'] = 0.0
# combined_ds['recession'] = d_perps
# combined_ds = lowV_ds
# combined_ds = highV_ds
combined_ds = combined_ds.assign_coords(
    theta_p_rads=np.radians(combined_ds.theta_p),
    theta_rads=np.radians(combined_ds.theta)
)#.drop(None)

In [47]:
combined_ds

<xarray.Dataset>
Dimensions:               (theta: 15, voltage: 1201)
Coordinates:
  * voltage               (voltage) float64 -29.98 -29.95 ... 1.665e-15 0.025
  * theta                 (theta) float64 2.0 3.0 4.0 5.0 ... 15.0 20.0 30.0
    theta_p               float64 0.0
    theta_p_rads          float64 0.0
    theta_rads            (theta) float64 0.03491 0.05236 ... 0.3491 0.5236
Data variables:
    str_iv_chi2           (theta) float64 723.8 802.2 ... 1.07e+03 1.067e+03
    str_iv_reduced_chi2   (theta) float64 0.6514 0.7195 0.738 ... 0.9598 0.9566
    str_iv_I_sat          (theta) float64 -14.44 -21.82 -29.39 ... -108.4 -150.9
    str_iv_d_I_sat        (theta) float64 0.01673 0.02617 ... 0.1702 0.2316
    str_iv_a              (theta) float64 0.005932 0.005801 ... 0.01262 0.008279
    str_iv_d_a            (theta) float64 0.0001485 0.0001528 ... 0.0001997
    str_iv_T_e            (theta) float64 1.113 1.099 1.089 ... 1.082 1.076
    str_iv_d_T_e          (theta) float64 0.005743 0.005632 ... 0.005819 0.00541
    str_iv_V_f            (theta) float64 -2.368 -2.338 -2.332 ... -2.666 -2.736
    str_iv_d_V_f          (theta) float64 0.003084 0.002892 ... 0.002625
    str_iv_run_dir        (theta) <U62 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-2.0' ... 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (theta) float64 -29.98 -29.98 -29.98 ... -29.98 -29.98
    str_iv_voltage_max    (theta) float64 -2.125 -2.025 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (theta) float64 1.106e+03 1.085e+03 ... 1.176e+03
    norm_iv_reduced_chi2  (theta) float64 1.003 0.9827 0.9814 ... 1.051 1.081
    norm_iv_I_sat         (theta) float64 -14.2 -21.51 -29.0 ... -107.3 -149.3
    norm_iv_d_I_sat       (theta) float64 0.01626 0.02425 ... 0.1448 0.2018
    norm_iv_a             (theta) float64 0.007396 0.007055 ... 0.01315 0.009172
    norm_iv_d_a           (theta) float64 0.0001542 0.0001512 ... 0.0001874
    norm_iv_voltage_min   (theta) float64 -27.61 -27.64 -27.65 ... -27.29 -27.25
    norm_iv_voltage_max   (theta) float64 -0.01163 -0.0166 ... -0.01911 -0.0208
    ion_chi2              (theta) float64 528.0 593.0 568.0 ... 820.2 776.8
    ion_reduced_chi2      (theta) float64 0.6118 0.6864 0.6566 ... 0.9649 0.916
    ion_I_sat             (theta) float64 14.46 21.91 29.43 ... 108.5 150.8
    ion_d_I_sat           (theta) float64 0.01913 0.03042 ... 0.2098 0.2797
    ion_a                 (theta) float64 0.005252 0.004938 ... 0.01178 0.007918
    ion_d_a               (theta) float64 0.0001623 0.0001699 ... 0.0002357
    ion_voltage_min       (theta) float64 3.839 3.842 3.834 ... 3.843 3.844
    ion_voltage_max       (theta) float64 12.05 12.06 12.06 ... 11.94 11.93
    current               (theta, voltage) float64 -15.88 -15.98 ... -0.0 nan
    current_e             (theta, voltage) float64 0.0 0.0 0.0 ... 0.0 nan
    current_i             (theta, voltage) float64 15.88 15.98 16.02 ... 0.0 nan
    d_current             (theta, voltage) float64 0.1944 0.2147 ... 0.0 nan
    d_current_e           (theta, voltage) float64 0.0 0.0 0.0 ... 7.594 0.0 nan
    d_current_i           (theta, voltage) float64 0.1944 0.2147 ... 0.0 nan
    time                  (theta, voltage) float64 0.0 0.06436 ... 5.294 nan

In [25]:
# combined_ds.to_netcdf('sheath_exp_fwp_ivs.nc')
# combined_ds.to_netcdf('se_lowV_tempscan_ivs.nc')
# combined_ds.to_netcdf('sheath_exp_hg_ivs.nc')
# combined_ds.to_netcdf('sheath_exp_fflwp_ivs.nc')
combined_ds.to_netcdf('sheath_exp_fwp-2_ivs.nc')

---
or, if loading, run this:

In [4]:
combined_ds = xr.open_dataset('sheath_exp_fwp_ivs.nc')

In [5]:
# sanity check

combined_ds.sel(theta=3.0, theta_p=0.0, recession=0.0).fit_run_dir

AttributeError: 'Dataset' object has no attribute 'fit_run_dir'

In [49]:
probe_desig = {
    'angled': {'theta_p':10.0, 'recession':0.0},
    'angled_semi-recessed': {'theta_p':10.0, 'recession':5.0e-4},
    'angled_recessed': {'theta_p':10.0, 'recession':1.0e-3},
    'semi-angled': {'theta_p':5.0, 'recession':0.0},
    'semi-angled_semi-recessed': {'theta_p':5.0, 'recession':5.0e-4},
    'semi-angled_recessed': {'theta_p':5.0, 'recession':1.0e-3},
    'flat': {'theta_p':0.0, 'recession':0.0},
    'flat_semi-recessed': {'theta_p':0.0, 'recession':5.0e-4},
    'flat_recessed': {'theta_p':0.0, 'recession':1.0e-3},
}

(2020-10-07) Everything above here has been made to make the netcdf with homogenised IV characteristics and several fits. The below was used to prototype these fitting routines and find the ones which worked best to get sheath expansion params out. 

This analysis is therefore carried on in a second notebook, with teh later parts of this spreadsheet left 'as is' for posterity and continued iteration. 

---

# Make some plots!



In [36]:
combined_ds.sel(theta_p=10.0, recession=0.0)

<xarray.Dataset>
Dimensions:               (theta: 9, voltage: 1204)
Coordinates:
    recession             float64 0.0
    theta_p_desc          <U6 'angled'
  * theta                 (theta) float64 3.0 4.0 5.0 6.0 ... 12.0 20.0 30.0
  * voltage               (voltage) float64 -29.97 -29.95 -29.93 ... 0.075 0.1
    theta_p               float64 10.0
    recession_desc        <U8 'flush'
    theta_p_rads          float64 0.1745
    theta_rads            (theta) float64 0.05236 0.06981 ... 0.3491 0.5236
Data variables:
    str_iv_chi2           (theta) float64 364.5 1.159e+03 ... 1.279e+03
    str_iv_reduced_chi2   (theta) float64 0.6839 1.039 2.215 ... 1.034 1.148
    str_iv_I_sat          (theta) float64 -29.62 -31.9 -34.09 ... -66.73 -91.63
    str_iv_d_I_sat        (theta) float64 1.108 0.05943 0.096 ... 0.1825 0.266
    str_iv_a              (theta) float64 0.03303 0.01186 ... 0.009779 0.006876
    str_iv_d_a            (theta) float64 0.01566 0.0002765 ... 0.0003706
    str_iv_T_e            (theta) float64 4.968 1.149 1.051 ... 1.001 1.054
    str_iv_d_T_e          (theta) float64 6.844 0.0032 ... 0.007241 0.01064
    str_iv_V_f            (theta) float64 1.29 -3.875 -3.816 ... -3.017 -2.708
    str_iv_d_V_f          (theta) float64 17.51 0.002991 ... 0.004249 0.005192
    str_iv_run_dir        (theta) object 'marconi/spice2/sheath_exp_fwp/angled_flush_as/alpha_yz_-3.0' ... 'marconi/spice2/sheath_exp_fwp/angled_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (theta) float64 -29.97 -29.97 -29.97 ... -29.97 -29.97
    str_iv_voltage_max    (theta) float64 -16.57 -2.025 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (theta) float64 372.6 852.8 ... 1.211e+03 1.247e+03
    norm_iv_reduced_chi2  (theta) float64 0.6965 0.8185 0.9512 ... 1.128 1.147
    norm_iv_I_sat         (theta) float64 -28.13 -31.28 -33.65 ... -67.17 -91.12
    norm_iv_d_I_sat       (theta) float64 0.1401 0.04547 ... 0.1648 0.2189
    norm_iv_a             (theta) float64 0.01544 0.01318 ... 0.008982 0.007277
    norm_iv_d_a           (theta) float64 0.0005605 0.0002129 ... 0.0003279
    norm_iv_voltage_min   (theta) float64 -28.97 -26.08 -26.2 ... -26.89 -27.25
    norm_iv_voltage_max   (theta) float64 -15.57 -0.007609 ... -0.01314 -0.02264
    ion_chi2              (theta) float64 372.6 893.9 ... 1.103e+03 1.241e+03
    ion_reduced_chi2      (theta) float64 0.6964 0.8579 0.8505 ... 1.028 1.142
    ion_I_sat             (theta) float64 28.13 31.03 33.35 ... 66.89 91.13
    ion_d_I_sat           (theta) float64 0.1401 0.0386 ... 0.1321 0.1857
    ion_a                 (theta) float64 0.01544 0.0143 ... 0.009507 0.007279
    ion_d_a               (theta) float64 0.0005605 0.0001896 ... 0.0002866
    ion_voltage_min       (theta) float64 7.84 0.02576 ... 0.03882 0.05837
    ion_voltage_max       (theta) float64 12.49 11.54 11.58 ... 11.81 11.93
    current               (theta, voltage) float64 -34.27 -34.26 ... 1.009e+03
    current_e             (theta, voltage) float64 0.0 0.0 ... -1.095e+03
    current_i             (theta, voltage) float64 34.27 34.26 ... 83.23 85.46
    d_current             (theta, voltage) float64 0.5479 0.5384 ... 7.908 10.45
    d_current_e           (theta, voltage) float64 0.0 0.0 0.0 ... 7.734 9.078
    d_current_i           (theta, voltage) float64 0.5479 0.5384 ... 1.736 2.263
    time                  (theta, voltage) float64 0.0 0.04279 ... 5.302 5.307

In [35]:
fig, ax = plt.subplots()
# combined_ds['fit_T_e'].plot(x='angle', linestyle='none', marker='.', mfc='none')
ax.errorbar('theta', 'fit_T_e', yerr='fit_d_T_e', data=combined_ds.sel(theta_p=10.0, recession=0.0), 
            linestyle='none', marker='.', mfc='none')

ValueError: The lengths of the data (7) and the error 9 do not match

In [36]:
fig, ax = plt.subplots(4, sharex=True, figsize=[4, 8])

plot_vars = ['fit_{}T_e', 'fit_{}I_sat', 'fit_{}a', 'fit_{}V_f']

for probe in combined_ds.probe.values:
    if probe == 'flat_recessed':
        continue
    plot_ds = combined_ds.sel(probe=probe)
    for i, plot_var in enumerate(plot_vars):
        ax[i].errorbar('theta', plot_var.format(''), yerr=plot_var.format('d_'), data=plot_ds, 
                       linestyle='none', marker='.', mfc='none', label=probe)
        ax[i].set_ylabel(r'${}$'.format(plot_var.split('{}')[-1]))

    # ax[3].plot('angle', 'fit_reduced_chi2', data=combined_ds, linestyle='none', marker='.', mfc='none')
    # ax[3].set_ylabel(r'$\chi_{\nu}^{2}$')
    ax[3].plot('theta', 'fit_voltage_max', data=plot_ds, linestyle='none', marker='.', mfc='none')

    ax[3].set_xlabel(r'$\theta$ [$\degree$]')
    
ax[0].legend()
ax[1].legend()
ax[2].legend()

fig.tight_layout()

AttributeError: 'Dataset' object has no attribute 'probe'

In [37]:
fig, ax = plt.subplots(2, sharex=True, figsize=[4, 8])

ax[0].plot('angle', 'fit_V_f', data=combined_ds, linestyle='none', marker='.', mfc='none')
ax[0].plot('angle', 'fit_voltage_max', data=combined_ds, linestyle='none', marker='.', mfc='none')

ax[1].plot(combined_ds.angle.values, (combined_ds['fit_voltage_max'] - combined_ds.fit_V_f).values, 
           linestyle='none', marker='.', mfc='none')
ax[1].axhline(y=3, **c.AX_LINE_DEFAULTS)

ValueError: Unrecognized character f in format string

In [38]:
fig, ax = plt.subplots(3, figsize=[6,10], sharex=True)

probe = 'flat'


voltage_range = np.array([-9.95, 0.1])

plot_ds = combined_ds.sortby('theta').sel(voltage=slice(*voltage_range), **probe_desig[probe])

plot_ds['current'].plot.line(x='voltage', hue='theta', ax=ax[0], linestyle='none', marker='+')
plot_ds['current_i'].plot.line(x='voltage', hue='theta', ax=ax[1])
plot_ds['current_e'].plot.line(x='voltage', hue='theta', ax=ax[2])

fitter = fts.FullIVFitter()
for theta in plot_ds.theta.values:
    ds = plot_ds.sel(theta=theta)
    dummy_voltage = np.linspace(ds.fit_voltage_min.values, ds.fit_voltage_max.values, 1000)
    fit_current = fitter.fit_function(dummy_voltage, 
                                      ds.fit_I_sat.values, 
                                      ds.fit_a.values, 
                                      ds.fit_T_e.values, 
                                      ds.fit_V_f.values)
    ax[0].plot(dummy_voltage, fit_current)
    ax[0].axvline(x=ds.fit_voltage_max.values, **c.AX_LINE_DEFAULTS)
    
ax[0].set_xlim(voltage_range[0] - 0.5, voltage_range[1] + 0.5)

fig.tight_layout()

AttributeError: 'Dataset' object has no attribute 'fit_voltage_min'

In [274]:
# The great Ion current plot with fits and a comparison to the fit to the total current IV

probe='flat'

fig = plt.figure(figsize=[10,7])
gs = fig.add_gridspec(2, 2)
ax0 = fig.add_subplot(gs[0:, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 1], sharex=ax1)
plt.setp(ax1.get_xticklabels(), visible=False)

ax = [ax0, ax1, ax2]

plot_ds = combined_ds.sel(**probe_desig[probe]).sortby('theta').sel(voltage=slice(-9.95, -6.0))

cols=[
    'probe',
    'angle',
    'I_sat',
    'd_I_sat',
    'a',
    'd_a',
    'reduced_chi2'
]
ion_fits = pd.DataFrame(columns=cols)

for theta in plot_ds.theta.values:
    sim_ds = plot_ds.sel(theta=theta)
    ax[0].errorbar(np.float_power(np.abs(sim_ds['voltage']), .75), 
                   sim_ds['current_i'], yerr=sim_ds['d_current_i'], 
                   label=r'${{\theta}} = $ {}'.format(theta))
    
    i_fitter = fts.IonCurrentSEFitter()
    fd = i_fitter.fit(np.float_power(np.abs(sim_ds['voltage'].values), .75), 
                      sim_ds['current_i'].values,
                      sigma=sim_ds['d_current_i'].values)
    
    ax[0].plot(*fd.get_fit_plottables(), color='grey')
    
    fd_dict = {k: v for k, v in fd.to_dict().items() 
               if k in cols}
    ion_fits = ion_fits.append({
        'probe': probe,
        'theta': theta,
        **fd_dict,
    }, ignore_index=True)

ax[0].set_ylabel(r'$\hat{I_{i}}$')
ax[0].set_xlabel(r'$|V|^{\frac{3}{4}}$')
ax[0].legend()

inner_ax = plt.axes([0.15, 0.45, .14, .2])
(-plot_ds['current_e']).plot.line(x='voltage', 
                                  hue='theta', 
                                  ax=inner_ax)
inner_ax.set_ylabel('')
inner_ax.set_xlabel('')
inner_ax.set_title(r'$I_e$')
inner_ax.legend().remove()

fig.suptitle(f'Ion Current Analysis for {probe} Probe')

ax[1].errorbar('theta', 'a', yerr='d_a', data=ion_fits, fmt='s', mfc='none', label=r'$I_{i}$')
ax[1].errorbar('theta', 'fit_a', yerr='fit_d_a', data=plot_ds, fmt='^', mfc='none', label=r'$I_{tot}$')
ax[1].errorbar(dummy_theta, calced_a, label=r'calc', fmt='-', 
               color='r', linewidth=0.8, alpha=0.6)
ax[1].legend()
ax[1].set_ylabel(r'$a$')
ax[1].set_ylim(1e-3, 5e-1)
ax[1].set_xlim(-0.5, 30.5)

ax[2].errorbar('theta', 'I_sat', yerr='d_I_sat', data=ion_fits, fmt='^', mfc='none', label=r'$I_{i}$')
ax[2].errorbar('theta', 'fit_I_sat', yerr='fit_d_I_sat', data=plot_ds, fmt='^', mfc='none', 
               label=r'$I_{tot}$')
ax[2].legend()
ax[2].set_ylabel(r'$I_{sat}$')
ax[2].set_xlabel(r'$\theta$')

gs.tight_layout(fig, rect=[0, 0, 1.0, 0.95])

No handles with labels found to put in legend.
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:79: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [260]:
# fig, ax = plt.subplots()

# ax.errorbar(sim_ds['voltage'], sim_ds['current_e'], yerr=sim_ds['d_current_e'], label='data')

# # for temperature in np.logspace(-0, 2, 3):
# #     ax.plot(sim_ds['voltage'], (0.0 - np.exp(-(-10.0 - sim_ds['voltage']) / temperature)), 
# #             label=f'{temperature}')
# ax.plot(sim_ds['voltage'], (0.0 - np.exp(-(-10.0 - sim_ds['voltage']) / 1.5)), 
#         label=f'{1.5}')
# ax.legend()

In [24]:
splopter =  spl.Splopter(spl_backup, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/angled_flush_as/alpha_yz_-12.0/backup_20201126-0947) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [25]:
parser = inp.InputParser(input_filename=spl_backup / 'input.inp')
denormaliser = nrm.Denormaliser(dt=splopter.tdata.dt, input_parser=parser) #, dimensions=3)
print(splopter.tdata.dt)
print(denormaliser.K)

[[0.00033962]]
55263494.10840861


In [27]:
theta = np.radians(10.0)

In [28]:
dV = parser.getfloat('mks', 'mks_te') #* c.BOLTZMANN / c.ELEM_CHARGE
T_e = parser.getfloat('mks', 'mks_te')
n_0 = parser.getfloat('mks', 'mks_n0')

L = parser.getfloat('rectangle2', 'yhigh') - parser.getfloat('rectangle2', 'ylow')
g = parser.getfloat('rectangle2', 'ylow') - parser.getfloat('rectangle0', 'yhigh') 
lambda_D = lpu.debye_length(T_e, n_0)
theta_p = np.radians(10.0)

I_0 = c.ELEM_CHARGE * n_0 * lpu.sound_speed(T_e) * L * lambda_D * np.sin(theta)

print(f"L = {L} L_d \n"
      f"g = {g} L_d \n"
      f"lambda_D = {lambda_D} \n"
      f"theta_p = {theta_p} \n"
      f"I_0 = {I_0}")

L = 300.0 L_d 
g = 60.0 L_d 
lambda_D = 1.6622799720325184e-05 
theta_p = 0.17453292519943295 
I_0 = 4.294003089384486


In [224]:
def decompose_new_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    y = (a * np.sqrt(np.sin(theta)) * (((L + g) * np.tan(theta)) 
                                       + (L * np.tan(theta_p)) - d_perp))
    x = np.tan(theta) + np.tan(theta_p)
    return x, y

def decompose_altnew_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    y = (a * np.sqrt(np.sin(theta)) * (((L + g) * np.tan(theta)) 
                                       + (L * np.tan(theta_p)) - d_perp))
    x = np.tan(theta) + np.tan(theta_p)
    return x, y

def decompose_sheath_exp_param(a, theta, L, g):
    y = a * (L + g) * np.sqrt(np.sin(theta))
    x = np.cos(theta) / np.sin(theta)
    return x, y


In [286]:
probe = 'flat'

plot_ds = combined_ds.sel(**probe_desig[probe]) #, theta=slice(1,10))

# array_1 = (plot_ds['fit_a'] * np.sqrt(np.sin(np.radians(plot_ds['theta']))) 
#            * (((L + g) * np.tan(np.radians(plot_ds['theta']))) + (L * np.tan(theta_p))))
# array_2 = np.tan(np.radians(plot_ds['theta'])) + np.tan(theta_p)
x, y = decompose_new_sheath_exp_param(plot_ds['fit_a'], plot_ds['theta_rads'], L, g, 
                                      plot_ds['recession'], plot_ds['theta_p_rads'])
x = x[~np.isnan(y)]
y = y[~np.isnan(y)]

x_1, y_1 = decompose_sheath_exp_param(plot_ds['fit_a'], plot_ds['theta_rads'], L, g)
x_1 = x_1[~np.isnan(y_1)]
y_1 = y_1[~np.isnan(y_1)]


sl_fitter = fts.StraightLineFitter()
sl_fit_data = sl_fitter.fit(x, y)
sl_fit_data_1 = sl_fitter.fit(x_1, y_1)

fig, ax = plt.subplots()
ax.plot(x, y, '.')
ax.plot(x_1, y_1, 'o')
ax.plot(*sl_fit_data.get_fit_plottables(), label=sl_fit_data.get_param_str())
ax.plot(*sl_fit_data_1.get_fit_plottables(), label=sl_fit_data_1.get_param_str())
ax.legend()



In [226]:
probe = 'angled'

plot_ds = combined_ds.sel(probe=probe) #, theta=slice(1,10))

array_x

sl_fitter = fts.StraightLineFitter()
sl_fit_data = sl_fitter.fit(array_x, array_y)

fig, ax = plt.subplots()
ax.plot(array_x, array_y, '.')
ax.plot(*sl_fit_data.get_fit_plottables(), label=sl_fit_data.get_param_str())
ax.legend()

ValueError: dimensions or multi-index levels ['probe'] do not exist

In [300]:
denormaliser(1, 'current')

1.9980219358841151

So getting weird negative slopes in these angle graphs, so I'm going to try denormalising properly and then trying the slightly altered fit from Bergmann's paper.

In [301]:
combined_ds['norm_current'] = denormaliser(combined_ds['current'], 'current')
combined_ds['norm_d_current'] = denormaliser(combined_ds['d_current'], 'current')

combined_ds['norm_current_i'] = denormaliser(combined_ds['current_i'], 'current')
combined_ds['norm_d_current_i'] = denormaliser(combined_ds['d_current_i'], 'current')

combined_ds['norm_current_e'] = denormaliser(combined_ds['current_e'], 'current')
combined_ds['norm_d_current_e'] = denormaliser(combined_ds['d_current_e'], 'current')

combined_ds.assign_coords(norm_voltage=denormaliser(combined_ds['voltage'], 'potential'))
# combined_ds['V'] = denormaliser(combined_ds['voltage'] - combined_ds['fit_V_f'], 'potential')


<xarray.Dataset>
Dimensions:           (recession: 3, theta: 6, theta_p: 3, voltage: 401)
Coordinates:
    theta_p_desc      (theta_p) <U11 'flat' 'semi-angled' 'angled'
  * voltage           (voltage) float64 -9.95 -9.9 -9.85 ... 9.95 10.0 10.05
  * theta             (theta) float64 1.0 2.0 3.0 5.0 10.0 30.0
    recession_desc    (recession) <U13 'flush' 'semi-recessed' 'recessed'
  * theta_p           (theta_p) float64 0.0 5.0 10.0
  * recession         (recession) float64 0.0 0.0005 0.001
    theta_p_rads      (theta_p) float64 0.0 0.08727 0.1745
    theta_rads        (theta) float64 0.01745 0.03491 0.05236 ... 0.1745 0.5236
    norm_voltage      (voltage) float64 -49.75 -49.5 -49.25 ... 49.75 50.0 50.25
Data variables:
    fit_chi2          (recession, theta_p, theta) float64 438.9 nan ... 150.7
    fit_reduced_chi2  (recession, theta_p, theta) float64 4.026 nan ... 1.077
    fit_I_sat         (recession, theta_p, theta) float64 -3.649 nan ... -72.9
    fit_d_I_sat       (recession, theta_p, theta) float64 0.5565 nan ... 1.781
    fit_a             (recession, theta_p, theta) float64 1.089e-20 ... 0.04027
    fit_d_a           (recession, theta_p, theta) float64 0.05543 ... 0.00634
    fit_T_e           (recession, theta_p, theta) float64 1.559 nan ... 0.9725
    fit_d_T_e         (recession, theta_p, theta) float64 0.2165 nan ... 0.04571
    fit_V_f           (recession, theta_p, theta) float64 -4.32 nan ... -2.764
    fit_d_V_f         (recession, theta_p, theta) float64 0.03581 ... 0.01809
    fit_run_dir       (recession, theta_p, theta) <U67 'marconi/spice2/sheath_exp/flat/alpha_yz_-1.0' ... 'marconi/spice2/sheath_exp/angled_recessed/alpha_yz_-30.0'
    fit_voltage_min   (recession, theta_p, theta) float64 -9.95 -9.95 ... -9.95
    fit_voltage_max   (recession, theta_p, theta) float64 -4.35 10.0 ... -2.8
    current           (recession, theta_p, theta, voltage) float64 -3.703 ... 925.9
    current_e         (recession, theta_p, theta, voltage) float64 -0.001758 ... -928.5
    current_i         (recession, theta_p, theta, voltage) float64 3.705 ... 2.579
    d_current         (recession, theta_p, theta, voltage) float64 0.08342 ... 6.024
    d_current_e       (recession, theta_p, theta, voltage) float64 0.001758 ... 6.075
    d_current_i       (recession, theta_p, theta, voltage) float64 0.08333 ... 0.4734
    time              (recession, theta_p, theta, voltage) float64 0.0 ... 2.581
    norm_current      (recession, theta_p, theta, voltage) float64 -7.399 ... 1.85e+03
    norm_d_current    (recession, theta_p, theta, voltage) float64 0.1667 ... 12.04
    norm_current_i    (recession, theta_p, theta, voltage) float64 7.402 ... 5.153
    norm_d_current_i  (recession, theta_p, theta, voltage) float64 0.1665 ... 0.9458
    norm_current_e    (recession, theta_p, theta, voltage) float64 -0.003512 ... -1.855e+03
    norm_d_current_e  (recession, theta_p, theta, voltage) float64 0.003512 ... 12.14
    V                 (voltage, recession, theta_p, theta) float64 -28.15 ... 64.07

In [228]:
dummy_theta = np.linspace(0.00001, 45.0, 5000)
calced_a = lpu.calc_sheath_expansion_coeff(
    T_e,
    n_0,
    L * lambda_D,
    g * lambda_D, 
    np.radians(dummy_theta),
    c_1=0.5,
    c_2=0.6,
)

alt_calced_a = lpu.calc_new_sheath_expansion_param(
    T_e,
    n_0,
    L * lambda_D,
    g * lambda_D, 
    np.radians(dummy_theta),
    0.0,
    np.radians(10.0),
    c_1=0.5,
    c_2=0.6,
)

In [338]:
# The altered Ion current plot

probe='flat'

fig = plt.figure(figsize=[10,6])
gs = fig.add_gridspec(2, 2)
ax0 = fig.add_subplot(gs[0:, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 1], sharex=ax1)
plt.setp(ax1.get_xticklabels(), visible=False)

ax = [ax0, ax1, ax2]

fig_iv, ax_iv = plt.subplots()

plot_ds = combined_ds.sel(**probe_desig[probe]).sortby('theta').sel(voltage=slice(-9.95, 0.0))

cols=[
    'probe',
    'angle',
    'I_sat',
    'd_I_sat',
    'a',
    'd_a',
    'reduced_chi2'
]
norm_ion_fits = pd.DataFrame(columns=cols)

for theta in plot_ds.theta.values:
    sim_ds = plot_ds.sel(theta=theta)
    
    V_f = fts.IVFitter.find_floating_pot(sim_ds['voltage'], sim_ds['current'])
    norm_V_f = denormaliser(V_f, 'potential')
    print(V_f)
    
    sim_ds = sim_ds.sel(voltage=slice(-9.95, V_f))
        
    ax[0].errorbar(np.float_power(np.abs(sim_ds['voltage'] - V_f), .75), 
                   sim_ds['current_i'], yerr=sim_ds['d_current_i'])
    
    i_fitter = fts.IonCurrentSEFitter()
    fd = i_fitter.fit(np.float_power(np.abs(sim_ds['voltage'] - V_f).values, .75), 
                      sim_ds['current_i'].values,
                      sigma=sim_ds['d_current_i'].values)
    
    ax[0].plot(*fd.get_fit_plottables())
    I_0 = fd.get_isat()
    
    ax_iv.errorbar(sim_ds['voltage'] - V_f, sim_ds['current']/I_0, yerr=sim_ds['d_current']/I_0, 
                   label=r'${{\theta}} = $ {}'.format(theta))
    ax_iv.legend()
    
    fd_dict = {k: v for k, v in fd.to_dict().items() 
               if k in cols}
    norm_ion_fits = norm_ion_fits.append({
        'probe': probe,
        'theta': theta,
        **fd_dict,
    }, ignore_index=True)

ax[0].set_ylabel(r'$I_{i}$')
ax[0].set_xlabel(r'$|V|^{\frac{3}{4}}$')

fig.suptitle(f'Ion Current Analysis for {probe.capitalize()} Probe')


ax[1].errorbar('theta', 'a', yerr='d_a', data=norm_ion_fits, fmt='s', mfc='none', label=r'$I_{i}$')
# ax[1].errorbar('theta', 'fit_a', yerr='fit_d_a', data=plot_ds, fmt='^', mfc='none', label=r'$I_{tot}$')
ax[1].errorbar(dummy_theta, calced_a, label=r'calc', fmt='-', 
               color='r', linewidth=0.8, alpha=0.6)
ax[1].errorbar(dummy_theta, alt_calced_a, label=r'calc - new', fmt='-', 
               color='orange', linewidth=0.8, alpha=0.6)
ax[1].legend()
ax[1].set_ylabel(r'$a$')
ax[1].set_ylim(1e-3, 5e-1)
ax[1].set_xlim(-0.5, 30.5)

ax[2].errorbar('theta', 'I_sat', yerr='d_I_sat', data=norm_ion_fits, fmt='^', mfc='none', label=r'$I_{i}$')
# ax[2].errorbar('theta', 'fit_I_sat', yerr='fit_d_I_sat', data=-plot_ds, fmt='^', mfc='none', 
#                label=r'$I_{tot}$')
ax[2].legend()
ax[2].set_ylabel(r'$I_{sat}$')
ax[2].set_xlabel(r'$\theta$')

gs.tight_layout(fig, rect=[0, 0, 1.0, 0.95])

-4.309590805138028
-4.828686741506997
-4.4956018612042445
-4.403934375163689
-4.0109978067524965
-2.9011683470668834


In [327]:
class PartialIVFitter(fts.IVFitter):
    """
    IV Fitter implementation utilising a partial, 4 parameter IV Curve fitting method.
    """
    def __init__(self):
        super().__init__()
        self._param_labels = {
            c.ION_SAT: 0,
            c.SHEATH_EXP: 1,
            c.ELEC_TEMP: 2,
        }
        self.default_values = (30.0, 0.0204, 1)
        self.default_bounds = (
            (-np.inf,      0,       0),
            ( np.inf,    1.5,  np.inf)
        )
        self.name = '4 Parameter Fit'

    def fit_function(self, v, *parameters):
        I_0 = parameters[self._param_labels[c.ION_SAT]]
        a = parameters[self._param_labels[c.SHEATH_EXP]]
        T_e = parameters[self._param_labels[c.ELEC_TEMP]]
        V = -v / T_e

        return I_0 * (1 - np.exp(-V) + np.where(v <= 0, (a * np.float_power(np.absolute(V), [0.75])), 0))

    def get_a_index(self):
        return self._param_labels[c.SHEATH_EXP]
    
class NormalisedIVFitter(fts.IVFitter):
    """
    IV Fitter implementation utilising a partial, 4 parameter IV Curve fitting method.
    """
    def __init__(self):
        super().__init__()
        self._param_labels = {
            c.ION_SAT: 0,
            c.SHEATH_EXP: 1,
        }
        self.default_values = (30.0, 0.0204)
        self.default_bounds = (
            (-np.inf,      0),
            ( np.inf,    1.5)
        )
        self.name = 'Normalised Parameter Fit'

    def fit_function(self, v, *parameters):
        I_0 = parameters[self._param_labels[c.ION_SAT]]
        a = parameters[self._param_labels[c.SHEATH_EXP]]
        
        return I_0 * (1 - np.exp(v) + np.where(v <= 0, (a * np.float_power(np.absolute(v), [0.75])), 0))

    def get_a_index(self):
        return self._param_labels[c.SHEATH_EXP]


In [330]:
# The denormalised IV Fitting

fig, ax = plt.subplots()

probe='flat'
plot_ds = combined_ds.sel(**probe_desig[probe]).sortby('theta').sel(voltage=slice(-9.95, 2.0))

cols=[
    'probe',
    'angle',
    'I_sat',
    'd_I_sat',
    'a',
    'd_a',
    'T_e',
    'd_T_e',
    'V_f',
    'd_V_f',
    'reduced_chi2'
]
iv_fits = pd.DataFrame(columns=cols)

calced_I_0s = []
fitted_I_0s = []

for theta in plot_ds.theta.values:
    sim_ds = plot_ds.sel(theta=theta)
    I_0 = c.ELEM_CHARGE * n_0 * lpu.sound_speed(T_e) * L * lambda_D * np.sin(np.radians(theta))
    calced_I_0s.append(I_0)
    
    V_f = fts.IVFitter.find_floating_pot(sim_ds['voltage'], sim_ds['current'])
    
    sim_ds = sim_ds.sel(voltage=slice(-9.95, V_f))
    
    voltage = sim_ds['voltage'].values - V_f
    current = sim_ds['current'].values
    d_current = sim_ds['d_current'].values
    
    iv_fitter = NormalisedIVFitter()
    fd = iv_fitter.fit(voltage, current, sigma=d_current)
    
    I_0 = fd.get_isat()
    fitted_I_0s.append(-I_0)
    
    ax.errorbar(voltage, current/I_0, yerr=d_current/I_0, fmt='x', label=f'{theta}')
    ax.plot(voltage, fd.fit_y / I_0, color='k')
    
    fd_dict = {k: v for k, v in fd.to_dict().items() 
               if k in cols}
    iv_fits = iv_fits.append({
        'probe': probe,
        'theta': theta,
        **fd_dict,
    }, ignore_index=True)

ax.legend(title=r'$\theta$')
ax.set_ylabel(r'$I / I_0$')
ax.set_xlabel(r'$V$')

Text(0.5, 0, '$V$')

In [306]:
fig, (ax, ax1) = plt.subplots(2, sharex=True)

ax.plot(plot_ds.theta.values, calced_I_0s, label='calced')
ax.plot(plot_ds.theta.values, fitted_I_0s, label='fitted')
ax.legend()

ax1.plot(plot_ds.theta.values, np.array(calced_I_0s) / np.array(fitted_I_0s))

NameError: name 'calced_I_0s' is not defined

In [344]:
# probe = 'angled'

plot_ds = combined_ds.sel(**probe_desig[probe]) #, theta=slice(1,10))

array_y = ion_fits['a'] * (L + g) * np.sqrt(np.sin(np.radians(ion_fits['theta'])))
array_x = np.cos(np.radians(ion_fits['theta'])) / np.sin(np.radians(ion_fits['theta']))

array_y_0 = norm_ion_fits['a'] * (L + g) * np.sqrt(np.sin(np.radians(norm_ion_fits['theta'])))
array_x_0 = np.cos(np.radians(norm_ion_fits['theta'])) / np.sin(np.radians(norm_ion_fits['theta']))

array_y_1 = plot_ds['fit_a'] * (L + g) * np.sqrt(np.sin(np.radians(plot_ds['theta'])))
array_x_1 = np.cos(np.radians(plot_ds['theta'])) / np.sin(np.radians(plot_ds['theta']))

array_y_2 = iv_fits['a'] * (L + g) * np.sqrt(np.sin(np.radians(iv_fits['theta'])))
array_x_2 = np.cos(np.radians(iv_fits['theta'])) / np.sin(np.radians(iv_fits['theta']))

array_y_old = flush_df['a'] * (L + g) * np.sqrt(np.sin(np.radians(flush_df['theta'])))
array_x_old = np.cos(np.radians(flush_df['theta'])) / np.sin(np.radians(flush_df['theta']))

array_y_calc = calced_a * (L + g) * np.sqrt(np.sin(np.radians(dummy_theta)))
array_x_calc = np.cos(np.radians(dummy_theta)) / np.sin(np.radians(dummy_theta))

# sl_fitter = fts.StraightLineFitter()
# sl_fit_data = sl_fitter.fit(array_x, array_y)
# sl_fit_data_1 = sl_fitter.fit(array_x_1, array_y_1)
# sl_fit_data_2 = sl_fitter.fit(array_x_2, array_y_2)


fig, ax = plt.subplots(1, 2, figsize=[12,6])

fig.suptitle('Sheath Expansion Fits to Flat-Flush Probe')

# ax[0].errorbar('theta', 'a', yerr='d_a', data=ion_fits, fmt='s', mfc='none', label=r'Ion Fit')
ax[0].errorbar('theta', 'a', yerr='d_a', data=norm_ion_fits, fmt='.', mfc='none', label=r'Norm Ion Fit')
# ax[0].errorbar('theta', 'fit_a', yerr='fit_d_a', data=plot_ds, fmt='o', mfc='none', 
#                label=r'IV Fit')
ax[0].errorbar('theta', 'a', yerr='d_a', data=iv_fits, fmt='^', mfc='none', label=r'Norm IV Fit')
ax[0].errorbar('theta', 'a', yerr='d_a', data=flush_df, fmt='*', mfc='none', label=r'Old Sims')

ax[0].errorbar(dummy_theta, calced_a, label=r'Theoretical', fmt='-', 
               color='r', linewidth=0.8, alpha=0.6)
# ax[0].errorbar(dummy_theta, alt_calced_a, label=r'calc - new', fmt='-', 
#                color='orange', linewidth=0.8, alpha=0.6)
ax[0].legend()
ax[0].set_ylabel(r'$a$')
ax[0].set_xlabel(r'$\theta$')
ax[0].set_ylim(-0.01, 5e-1)
ax[0].set_xlim(-0.5, 30.5)


# ax[1].plot(array_x, array_y, 's', mfc='none', label='Ion fit')
ax[1].plot(array_x_0, array_y_0, '.', mfc='none', label='Norm Ion fit')
# ax[1].plot(array_x_1, array_y_1, 'o', mfc='none', label='IV Fit')
ax[1].plot(array_x_2, array_y_2, '^', mfc='none', label='Norm IV Fit')
ax[1].plot(array_x_old, array_y_old, '*', mfc='none', label='Old sim')

ax[1].plot(array_x_calc, array_y_calc, '-', color='r', linewidth=0.8, alpha=0.6, label='Theoretical')

# ax[1].plot(*sl_fit_data.get_fit_plottables(), label=sl_fit_data.get_param_str())
# ax[1].plot(*sl_fit_data_1.get_fit_plottables(), label=sl_fit_data_1.get_param_str())
# ax[1].plot(*sl_fit_data_2.get_fit_plottables(), label=sl_fit_data_2.get_param_str())
ax[1].legend()

ax[1].set_xlabel(r'$\cos{\theta} \; / \; \sin{\theta}$')
ax[1].set_ylabel(r'$a L \sin^{1/2}{\theta}$')
ax[1].set_xlim(-0.5, 60)
ax[1].set_ylim(-0.5, 40)

fig.tight_layout(rect=[0, 0, 1.0, 0.95])

## Compare also to some older simulations

In [153]:
lowdens_dir = pth.Path('/home/jleland/data/external/spice/')
fit_data_filename = 'lowdens_fitdata.csv'

fit_df = pd.read_csv(lowdens_dir / fit_data_filename, index_col=0)
fit_df

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,angled,1.0,1.025535,0.020026,3.283136,0.082634,0.034400,0.008213,-3.372417,0.007165
1,angled,2.0,1.045895,0.016761,7.148155,0.163483,0.013990,0.007510,-3.526380,0.006954
2,angled,3.0,1.048542,0.008816,11.100458,0.129524,0.004756,0.003758,-3.484159,0.003447
3,angled,4.0,1.023927,0.018306,14.362077,0.343829,0.012245,0.007634,-3.423099,0.007020
4,angled,5.0,1.013085,0.010332,18.032248,0.256156,0.010595,0.004569,-3.457255,0.004239
...,...,...,...,...,...,...,...,...,...,...
83,sprobe,30.0,1.033611,0.010399,62.740506,0.552424,0.012586,0.002549,-2.732093,0.002039
84,sprobe,45.0,1.047113,0.014941,83.666200,1.018863,0.007753,0.003510,-2.690400,0.002807
85,sprobe,60.0,1.006582,0.010704,86.495626,0.719475,0.016005,0.002336,-2.594291,0.002069
86,sprobe,75.0,0.996101,0.012401,87.772275,0.795372,0.017997,0.002498,-2.483807,0.002520


In [154]:
fit_df.groupby('probe').groups

{'angled': Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='int64'),
 'flush': Int64Index([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
             35, 36, 37, 38],
            dtype='int64'),
 'rearwall': Int64Index([39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53], dtype='int64'),
 'recessed': Int64Index([54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
             71, 72],
            dtype='int64'),
 'sprobe': Int64Index([73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87], dtype='int64')}

In [155]:
flush_df = fit_df.loc[fit_df['probe'] == 'flush']
flush_df = flush_df.loc[flush_df['angle'] <= 30]
# flush_df = flush_df.loc[flush_df['angle'] > 3]
flush_df = flush_df.drop(20)
flush_df = flush_df.rename(columns={'angle':'theta'})
flush_df

,probe,theta,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
18,flush,1.0,1.196922,0.159300,2.005588,0.248067,0.234585,0.045250,-2.824577,0.023557
19,flush,2.0,0.746410,0.041318,3.856090,0.219110,0.189780,0.018324,-2.920495,0.018052
21,flush,4.0,0.862937,0.021906,8.867943,0.236408,0.130579,0.008518,-3.000390,0.007916
22,flush,5.0,0.984287,0.023889,12.246093,0.301755,0.088044,0.007808,-2.990635,0.006228
23,flush,6.0,0.980118,0.021951,14.471464,0.340238,0.076094,0.007440,-3.051633,0.006168
24,flush,7.0,0.940242,0.009109,16.238058,0.168016,0.075870,0.003233,-3.054091,0.002874
25,flush,8.0,1.017323,0.009001,19.388100,0.176069,0.055888,0.002833,-3.014263,0.002248
26,flush,9.0,0.984877,0.007602,21.051741,0.187058,0.060599,0.002850,-3.167852,0.002429
27,flush,10.0,1.011952,0.009473,23.709109,0.223009,0.052364,0.002896,-2.981223,0.002314
28,flush,15.0,1.055231,0.012629,36.369258,0.426445,0.030411,0.003559,-2.932875,0.002739


In [156]:
angled_df = fit_df.loc[fit_df['probe'] == 'angled']
angled_df = angled_df.loc[angled_df['angle'] <= 30]
# flush_df = flush_df.loc[flush_df['angle'] > 3]
# angled_df = angled_df.drop(20)
angled_df = angled_df.rename(columns={'angle':'theta'})
angled_df

,probe,theta,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,angled,1.0,1.025535,0.020026,3.283136,0.082634,3.440025e-02,0.008213,-3.372417,0.007165
1,angled,2.0,1.045895,0.016761,7.148155,0.163483,1.398953e-02,0.007510,-3.526380,0.006954
2,angled,3.0,1.048542,0.008816,11.100458,0.129524,4.755583e-03,0.003758,-3.484159,0.003447
3,angled,4.0,1.023927,0.018306,14.362077,0.343829,1.224509e-02,0.007634,-3.423099,0.007020
4,angled,5.0,1.013085,0.010332,18.032248,0.256156,1.059543e-02,0.004569,-3.457255,0.004239
5,angled,9.0,1.051815,0.008428,32.520786,0.336018,9.930011e-03,0.003315,-3.362492,0.002789
6,angled,10.0,1.033696,0.010810,35.128063,0.458634,1.811731e-02,0.004147,-3.335452,0.003589
7,angled,15.0,1.088697,0.025033,54.814766,1.440419,4.923012e-18,0.008144,-3.217063,0.006442
8,angled,20.0,1.067608,0.011998,64.130095,0.747523,1.280170e-02,0.003548,-3.038085,0.002769
9,angled,25.0,1.058848,0.009242,75.869397,0.611659,1.295753e-02,0.002377,-2.838315,0.001848


In [157]:
fig, ax = plt.subplots()
ax.errorbar('theta', 'a', yerr='d_a', data=flush_df, fmt='x')
ax.errorbar('theta', 'a', yerr='d_a', data=angled_df, fmt='s', mfc='none')

ax.errorbar(dummy_theta, calced_a, label=r'Theoretical', fmt='-', 
            color='r', linewidth=0.8, alpha=0.6)
ax.errorbar(dummy_theta, alt_calced_a, label=r'calc - new', fmt='-', 
            color='orange', linewidth=0.8, alpha=0.6)

ax.set_ylim(0, 5e-1)
ax.set_xlim(-0.5, 30.5)

(-0.5, 30.5)

In [159]:
plot_ds = combined_ds.sel(**probe_desig[probe]) #, theta=slice(1,10))

array_y = flush_df['a'] * (L) * np.sqrt(np.sin(np.radians(flush_df['theta'])))
array_x = np.cos(np.radians(flush_df['theta'])) / np.sin(np.radians(flush_df['theta']))

array_y_calc = calced_a * (L) * np.sqrt(np.sin(np.radians(dummy_theta)))
array_x_calc = np.cos(np.radians(dummy_theta)) / np.sin(np.radians(dummy_theta))

# sl_fitter = fts.StraightLineFitter()
# sl_fit_data = sl_fitter.fit(array_x, array_y)
# sl_fit_data_1 = sl_fitter.fit(array_x_1, array_y_1)
# sl_fit_data_2 = sl_fitter.fit(array_x_2, array_y_2)


fig, ax = plt.subplots(1, 2, figsize=[12,6])

fig.suptitle('Sheath Expansion Fits to Flat-Flush Probe')

ax[0].errorbar('theta', 'a', yerr='d_a', data=flush_df, fmt='s', mfc='none', label=r'Old Sim Fit')

ax[0].errorbar(dummy_theta, calced_a, label=r'Theoretical', fmt='-', 
               color='r', linewidth=0.8, alpha=0.6)
# ax[0].errorbar(dummy_theta, alt_calced_a, label=r'calc - new', fmt='-', 
#                color='orange', linewidth=0.8, alpha=0.6)
ax[0].legend()
ax[0].set_ylabel(r'$a$')
ax[0].set_xlabel(r'$\theta$')
ax[0].set_ylim(0, 5e-1)
ax[0].set_xlim(-0.5, 30.5)


ax[1].plot(array_x, array_y, 's', mfc='none', label='Old sim fit')

ax[1].plot(array_x_calc, array_y_calc, '-', color='r', linewidth=0.8, alpha=0.6, label='Theoretical')

# ax[1].plot(*sl_fit_data.get_fit_plottables(), label=sl_fit_data.get_param_str())
# ax[1].plot(*sl_fit_data_1.get_fit_plottables(), label=sl_fit_data_1.get_param_str())
# ax[1].plot(*sl_fit_data_2.get_fit_plottables(), label=sl_fit_data_2.get_param_str())
ax[1].legend()

ax[1].set_xlabel(r'$\cos{\theta} \; / \; \sin{\theta}$')
ax[1].set_ylabel(r'$a L \sin^{1/2}{\theta}$')
ax[1].set_xlim(-0.5, 60)
ax[1].set_ylim(-0.55, 35)

fig.tight_layout(rect=[0, 0, 1.0, 0.95])

# Making the IV characteristic plot in Bergmann's 1994 paper

In [247]:
def a_0(theta):
    return lpu.calc_sheath_expansion_coeff(
        T_e, 
        n_0, 
        L * lambda_D, 
        g * lambda_D, 
        np.radians(theta),
        c_1=0.5, 
        c_2=0.6
    )

In [249]:
fig, ax = plt.subplots()
dummy_voltage_2 = np.linspace(-30, 0, 9950)

I_0 = c.ELEM_CHARGE * n_0 * lpu.sound_speed(T_e) * L * lambda_D * np.sin(np.radians(90.0))

piv_fitter = PartialIVFitter()
dummy_current = piv_fitter.fit_function(dummy_voltage_2, *[I_0, 0.25, 1.0])

ax.plot(dummy_voltage_2, -piv_fitter.fit_function(dummy_voltage_2, *[I_0, a_0(90.0), 1.0]), 
        color='k', linestyle='--', label=f'Ideal')
for theta in [30.0, 10.0, 5.0, 3.0, 2.0, 1.0]:
    ax.plot(dummy_voltage_2, -piv_fitter.fit_function(dummy_voltage_2, *[I_0, a_0(theta), 1.0]), 
            label=r'$\theta = {}$'.format(theta))
ax.legend()
# ax.set_ylim(-4.0,0.0)
ax.set_ylim(-100.0,0.0)
ax.set_xlim(-30.0,0.0)

(-30.0, 0.0)

## Probe Sanity Checks

Looking into why the current is not coming out properly. 

In [6]:
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

In [20]:
mag_probes = lpu.MagnumProbes()
mag_probes['L'].d_perp

(0.0003,)

In [21]:
flush_square = lpu.AngledTipProbe(a=5e-3, b=5e-3, L=5e-3, g=1e-3, d_perp=0e-4, 
                                  theta_f=0.0, theta_p=0.0)

In [29]:
alpha = np.radians(10.0)
print(f'get_collection_area() = \t {flush_square.get_collection_area(alpha)}')
print(f'get_2d_collection_length() = \t {flush_square.get_2d_collection_length(alpha)}')
print(f'(L + g) * sin(0) = \t\t {6e-3 * np.sin(alpha)}')
print(f'calc_exposed_lengths() = \t {flush_square.calc_exposed_lengths(alpha)}')
print(f'get_isat() = \t\t\t {flush_square.get_isat(T_e, n_0, alpha)}')

get_collection_area() = 	 [5.20944533e-06]
get_2d_collection_length() = 	 [0.00103925]
(L + g) * sin(0) = 		 0.001041889066001582
calc_exposed_lengths() = 	 (array([0.]), array([0.00017365]))
get_isat() = 			 [0.02583201]


In [109]:
probe_isat = mag_probes['L'].get_isat(T_e, n_0, alpha)
probe_current_2d = lpu.sound_speed(T_e) * n_0 * c.ELEM_CHARGE * ((L + g) * lambda_D) * np.sin(alpha)

print(f'I_sat = {probe_isat} \n'
      f'I_probe = {probe_current_pul} \n'
      f'I_sat/I_probe = {probe_isat/probe_current_2d} \n')

I_sat = [0.02107866] 
I_probe = 5.152803707261383 
I_sat/I_probe = [0.00409072] 



In [54]:
flush_square.get_isat(T_e, n_0, alpha) / probe_current_2d

array([0.00501319])

In [55]:
probe_current_2d * flush_square.a

0.025764018536306915

In [46]:
flat_flush_ds = combined_ds.sel(**probe_desig['flat'], theta=10.0)

In [64]:
fitter = fts.FullIVFitter()

fig, ax = plt.subplots()
flat_flush_ds['current'].plot.line(x='voltage', ax=ax)

I_sat_sim = flat_flush_ds['str_iv_I_sat'].values

ax.axhline(y=I_sat_sim, **c.AX_LINE_DEFAULTS)


In [88]:
print(f'dt = {splopter.tdata.dt}, K = {denormaliser.K:.3g}')

dt = [[0.00033962]], K = 5.53e+07


In [60]:
denormaliser_3d = nrm.Denormaliser(dt=splopter.tdata.dt, input_parser=parser, dimensions=3)

In [106]:
print(denormaliser(1, 'current'))
print(denormaliser_3d(1, 'current'))

0.19980219358841153
3.321271847701805e-05


In [111]:
denormaliser(I_sat_sim, 'current') * flush_square.a

-0.028757416409148275

In [72]:
probe_current_2d / -I_sat_sim 

0.17900451646862947

In [85]:
(denormaliser.K * parser.getfloat('geom', 'npc')) / (lpu.debye_length(5, 1e18))**2

1e+19

In [104]:
K = (1e18 * lpu.debye_length(5, 1e18)**2) / 50
print(f'{K:.3g}')

5.53e+06


In [90]:
parser.get_commented_params()

{'n_e': 1e+18, 'T_e': 5.0, 'B': 0.8, 'L_probe': 5.0}

In [97]:
denormaliser.debye_length

1.6622799720325184e-05

In [95]:
lpu.debye_length(5, 1e18)

1.6622799720325184e-05

In [110]:
denormaliser.K

5526349.410840861

In [98]:
((denormaliser.simulation_params[c.ELEC_DENS] * denormaliser.debye_length**denormaliser.dimensions)
 / float(parser.get(c.INF_SEC_GEOMETRY, c.INF_PART_PER_CELL)[:-1]))

55263494.10840861

In [103]:
parser.getfloat(c.INF_SEC_GEOMETRY, c.INF_PART_PER_CELL)

50.0

FOUND IT! 

A bug with how the number of particles per cell was being retrieved in the denormaliser. 